In [ ]:
# Use the following features to derive new features:
# lang (en, und, tt) ... convert to a bool for each language
# to_user_name -> bool for conversational tweet
# from_user_lang .. alternative to loang ()probably not needed)
# source -> contains "instagram.com" becomes is_instagram
# tweet_text_contains_url -> needs a regex (google) or just http?
# also need to filter out null LGA as they are useless to us
# then need to summarise the new features per LGA

# SEPARATE TASK : Improve smc_tokenizer punct, words ending in !

In [94]:
import numpy as np
import pandas as pd
import os
import re
base = "../../data/clean/"
pd.set_option('display.max_rows', 200)

In [95]:
tweetdf = pd.read_csv(base+'filtered_tweets.csv')
tweetdf = tweetdf.loc[~pd.isnull(tweetdf['lga'])].reset_index()
len(tweetdf)

50710

In [96]:
# Language booleans
langdf = tweetdf.lang.str.get_dummies()
langs = langdf.columns.tolist()
langdf = pd.concat([tweetdf,langdf],axis=1)[['lga']+langs]
print(langdf)

                    lga  ar  bn  ca  cs  cy  da  de  en  es ...  ro  ru  sl  \
0                Sydney   0   0   0   0   0   0   0   1   0 ...   0   0   0   
1            Inner West   0   0   0   0   0   0   0   1   0 ...   0   0   0   
2                  Ryde   0   0   0   0   0   0   0   1   0 ...   0   0   0   
3             Blacktown   0   0   0   0   0   0   0   1   0 ...   0   0   0   
4                  Ryde   0   0   0   0   0   0   0   1   0 ...   0   0   0   
5              Waverley   0   0   0   0   0   0   0   1   0 ...   0   0   0   
6              Waverley   0   0   0   0   0   0   0   0   0 ...   0   0   0   
7              Randwick   0   0   0   0   0   0   0   1   0 ...   0   0   0   
8                Sydney   0   0   0   0   0   0   0   1   0 ...   0   0   0   
9      Northern Beaches   0   0   0   0   0   0   0   1   0 ...   0   0   0   
10              Bayside   0   0   0   0   0   0   0   1   0 ...   0   0   0   
11     Northern Beaches   0   0   0   0   0   0   0 

In [97]:
DROP_COUNT=len(tweetdf) / 500
lga_langdf = langdf.groupby(['lga']).sum().reset_index()
sums = langdf.drop('lga',axis=1).sum().to_dict()
to_drop = [k for (k,v) in sums.items() if k == 'en' or v < DROP_COUNT]
lga_langdf = lga_langdf.drop(to_drop, axis=1)
lga_langdf

,lga,ar,cy,de,es,fr,ht,in,it,ja,pt,tl,und,zh
0,Bayside,4,8,8,10,6,1,22,1,59,16,7,45,6
1,Blacktown,0,2,4,3,4,1,3,1,8,3,6,14,0
2,Blue Mountains,0,0,0,0,1,0,1,0,0,1,1,0,0
3,Burwood,0,0,0,0,0,0,1,0,2,0,0,4,2
4,Camden,0,4,3,0,2,0,3,0,2,0,1,3,0
5,Campbelltown,0,9,0,1,1,0,3,0,0,1,1,4,0
6,Canada Bay,1,0,2,3,4,1,1,1,0,3,2,6,4
7,Canterbury-Bankstown,1,0,1,2,4,1,6,5,1,0,1,53,1
8,Central Coast,0,0,0,0,0,0,1,0,0,0,1,1,0
9,Cumberland,1,1,6,2,1,0,55,0,3,1,5,9,2


Language counts will probably not be useful unless converted to per_capita or percentage. As such we would probalby need ot choose specific languages of interest rather than dynmically choosing them as in the code above. Furthermore the distribution across LGAs does not seem likely to be predictive, with the possibel exception of "und" (undetermined). Given there is also a risk this will be considered to be a form of ethnic profiling it is probably safer to leave these features out.

In [98]:
replies_df = tweetdf[['lga','to_user_name']].groupby(['lga']).agg({'to_user_name': ['count']}).reset_index()
replies_df.columns = ['lga','total_reply_tweets']
replies_df

,lga,total_reply_tweets
0,Bayside,35
1,Blacktown,9
2,Blue Mountains,1
3,Burwood,0
4,Camden,0
5,Campbelltown,1
6,Canada Bay,12
7,Canterbury-Bankstown,5
8,Central Coast,0
9,Cumberland,9


In [104]:
lang_ind_df = pd.read_csv("../../data/clean/language_independent_features.csv")
lang_ind_df

,lga,total_tweets,total_tweeters,population,tweets_per_capita,tweeters_per_capita
0,Bayside,1752,894,156058,0.011227,0.005729
1,Blacktown,948,321,336962,0.002813,0.000953
2,Blue Mountains,141,57,76904,0.001833,0.000741
3,Burwood,139,86,36809,0.003776,0.002336
4,Camden,250,50,78218,0.003196,0.000639
5,Campbelltown,276,78,157006,0.001758,0.000497
6,Canada Bay,431,191,88015,0.004897,0.002170
7,Canterbury-Bankstown,549,226,346302,0.001585,0.000653
8,Central Coast,61,31,327736,0.000186,0.000095
9,Cumberland,828,165,216079,0.003832,0.000764


In [105]:
lang_ind_df = pd.merge(lang_ind_df, lga_langdf[['lga','und']],on='lga')
lang_ind_df['und_ratio'] = lang_ind_df['und'] / lang_ind_df['total_tweets']
lang_ind_df = pd.merge(lang_ind_df, replies_df[['lga','total_reply_tweets']],on='lga')
lang_ind_df['reply_ratio'] = lang_ind_df['total_reply_tweets'] / lang_ind_df['total_tweets']
lang_ind_df['replies_per_capita'] = lang_ind_df['total_reply_tweets'] / lang_ind_df['population']
lang_ind_df

,lga,total_tweets,total_tweeters,population,tweets_per_capita,tweeters_per_capita,und,und_ratio,total_reply_tweets,reply_ratio,replies_per_capita
0,Bayside,1752,894,156058,0.011227,0.005729,45,0.025685,35,0.019977,0.000224
1,Blacktown,948,321,336962,0.002813,0.000953,14,0.014768,9,0.009494,0.000027
2,Blue Mountains,141,57,76904,0.001833,0.000741,0,0.000000,1,0.007092,0.000013
3,Burwood,139,86,36809,0.003776,0.002336,4,0.028777,0,0.000000,0.000000
4,Camden,250,50,78218,0.003196,0.000639,3,0.012000,0,0.000000,0.000000
5,Campbelltown,276,78,157006,0.001758,0.000497,4,0.014493,1,0.003623,0.000006
6,Canada Bay,431,191,88015,0.004897,0.002170,6,0.013921,12,0.027842,0.000136
7,Canterbury-Bankstown,549,226,346302,0.001585,0.000653,53,0.096539,5,0.009107,0.000014
8,Central Coast,61,31,327736,0.000186,0.000095,1,0.016393,0,0.000000,0.000000
9,Cumberland,828,165,216079,0.003832,0.000764,9,0.010870,9,0.010870,0.000042


In [106]:
# source -> contains "instagram.com" becomes is_instagram
instadf = tweetdf[tweetdf.source.str.contains('http://instagram\.com')]
lga_instadf = instadf[['lga','source']].groupby(['lga']).agg({'source': ['count']}).reset_index()
lga_instadf.columns = ['lga','total_instagrams']
lga_instadf

,lga,total_instagrams
0,Bayside,1103
1,Blacktown,648
2,Blue Mountains,107
3,Burwood,117
4,Camden,108
5,Campbelltown,151
6,Canada Bay,319
7,Canterbury-Bankstown,356
8,Central Coast,37
9,Cumberland,301


In [107]:
lga_instagrammers_df = instadf[['lga','from_user_name']].groupby(['lga']).agg({'from_user_name': lambda x: x.nunique()}).reset_index()
lga_instagrammers_df.columns = ['lga','total_instagrammers']
lga_instagrammers_df

,lga,total_instagrammers
0,Bayside,643
1,Blacktown,289
2,Blue Mountains,46
3,Burwood,72
4,Camden,42
5,Campbelltown,64
6,Canada Bay,163
7,Canterbury-Bankstown,179
8,Central Coast,26
9,Cumberland,131


In [108]:
linkdf = tweetdf[tweetdf.text.str.contains('http')]
lga_linkdf = linkdf[['lga','id']].groupby(['lga']).count().reset_index()
lga_linkdf.columns = ['lga','total_link_tweets']
lga_linkdf

,lga,total_link_tweets
0,Bayside,1704
1,Blacktown,857
2,Blue Mountains,134
3,Burwood,139
4,Camden,246
5,Campbelltown,271
6,Canada Bay,420
7,Canterbury-Bankstown,531
8,Central Coast,61
9,Cumberland,391


In [109]:
lang_ind_df = pd.merge(lang_ind_df, lga_instadf,on='lga')
lang_ind_df = pd.merge(lang_ind_df, lga_instagrammers_df,on='lga')
lang_ind_df = pd.merge(lang_ind_df, lga_linkdf,on='lga')
lang_ind_df['instagram_ratio'] = lang_ind_df['total_instagrams'] / lang_ind_df['total_tweets']
lang_ind_df['instagrammers_ratio'] = lang_ind_df['total_instagrammers'] / lang_ind_df['total_tweeters']
lang_ind_df['link_tweets_ratio'] = lang_ind_df['total_link_tweets'] / lang_ind_df['total_tweets']
lang_ind_df

,lga,total_tweets,total_tweeters,population,tweets_per_capita,tweeters_per_capita,und,und_ratio,total_reply_tweets,reply_ratio,replies_per_capita,total_instagrams,total_instagrammers,total_link_tweets,instagram_ratio,instagrammers_ratio,link_tweets_ratio
0,Bayside,1752,894,156058,0.011227,0.005729,45,0.025685,35,0.019977,0.000224,1103,643,1704,0.629566,0.719239,0.972603
1,Blacktown,948,321,336962,0.002813,0.000953,14,0.014768,9,0.009494,0.000027,648,289,857,0.683544,0.900312,0.904008
2,Blue Mountains,141,57,76904,0.001833,0.000741,0,0.000000,1,0.007092,0.000013,107,46,134,0.758865,0.807018,0.950355
3,Burwood,139,86,36809,0.003776,0.002336,4,0.028777,0,0.000000,0.000000,117,72,139,0.841727,0.837209,1.000000
4,Camden,250,50,78218,0.003196,0.000639,3,0.012000,0,0.000000,0.000000,108,42,246,0.432000,0.840000,0.984000
5,Campbelltown,276,78,157006,0.001758,0.000497,4,0.014493,1,0.003623,0.000006,151,64,271,0.547101,0.820513,0.981884
6,Canada Bay,431,191,88015,0.004897,0.002170,6,0.013921,12,0.027842,0.000136,319,163,420,0.740139,0.853403,0.974478
7,Canterbury-Bankstown,549,226,346302,0.001585,0.000653,53,0.096539,5,0.009107,0.000014,356,179,531,0.648452,0.792035,0.967213
8,Central Coast,61,31,327736,0.000186,0.000095,1,0.016393,0,0.000000,0.000000,37,26,61,0.606557,0.838710,1.000000
9,Cumberland,828,165,216079,0.003832,0.000764,9,0.010870,9,0.010870,0.000042,301,131,391,0.363527,0.793939,0.472222


In [88]:
alldf = pd.read_csv(base+'tweets_w_lga.csv')
lgadf = tweetdf.loc[~pd.isnull(tweetdf['lga'])].reset_index()
latdf = tweetdf.loc[~pd.isnull(tweetdf['lat'])].reset_index()
print(len(lgadf),len(latdf))

75082 75082


In [90]:
len(alldf[alldf.lga == 'Sydney'])

34116

In [116]:
cols = ['a','b','c']
cols.remove('b')
cols[:0] = 'b'
cols

['b', 'a', 'c']